In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.2/934.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.6/275.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.7 MB/s eta 0:00:

In [ ]:
from darts.models.forecasting.dlinear import DLinearModel
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

In [ ]:
processed_data_dir = '/content/drive/MyDrive/M5-A1/2. data/processed/'
BASE     = processed_data_dir+'grid_part_1.pkl'
PRICE    = processed_data_dir+'grid_part_2.pkl'
CALENDAR = processed_data_dir+'grid_part_3.pkl'
ver, KKK = 'priv', 0
START_TRAIN = 41
END_TRAIN   = 1941 - 28*KKK
P_HORIZON   = 28
USE_AUX     = False
TARGET = 'sales'
LAGS     = processed_data_dir+'lags_df_28.pkl'
MEAN_ENC = processed_data_dir+'mean_encoding_df.pkl'

remove_features = ['id','cat_id', 'state_id','store_id','dept_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_item_id_store_id_mean','enc_item_id_store_id_std']

In [ ]:
processed_data_dir_base = '/content/drive/MyDrive/M5-thesis/M5-FITS/processed/'
global_df = pd.read_pickle(processed_data_dir_base + 'raw_data.pkl')

In [ ]:
global_df.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'release', 'sell_price', 'price_max', 'price_min', 'price_std',
       'price_mean', 'price_norm', 'price_nunique', 'item_nunique',
       'price_momentum', 'price_momentum_m', 'price_momentum_y',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y',
       'tm_wm', 'tm_dw', 'tm_w_end'],
      dtype='object')

In [ ]:
from darts.timeseries import TimeSeries

TRAIN_START = 0
TEST_START = 1941
HORIZON = 28
VER = 2.0
stores = global_df['store_id'].unique()
depts = global_df['dept_id'].unique()
stores, depts

(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
 Categories (10, object): ['CA_1', 'CA_2', 'CA_3', 'CA_4', ..., 'TX_3', 'WI_1', 'WI_2', 'WI_3'],
 ['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1', 'FOODS_2', 'FOODS_3']
 Categories (7, object): ['FOODS_1', 'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1',
                          'HOUSEHOLD_2'])

In [ ]:
"""
get df_train and features
"""
def get_data_by_store_id(store_id, dept_id):
  df = global_df.copy()

  store_data = df[(df['store_id'] == store_id) & (df['dept_id'] == dept_id) & (df['d'] >= TRAIN_START)]
  df_train = store_data.pivot(index='id', columns='d', values='sales')
  df_train.fillna(0, inplace=True)
  df_calendar = store_data.groupby(['d']).agg({
    'event_name_1': 'first',
    'event_name_2': 'first',
    'snap_CA': 'first',
    'snap_TX': 'first',
    'snap_WI': 'first',
    'tm_w_end': 'first',
    'tm_dw': 'first',
    'sell_price': 'first',
    'tm_w': 'first',
    'tm_m': 'first',

  }).reset_index()
  df_calendar.index = df_calendar['d']

  del df, store_data
  gc.collect()

  return df_train, df_calendar

In [ ]:
import math

class M5DLinear(object):
  def __init__(self, horizon, df_train, df_calendar, epochs, lookback):
    self.horizon = horizon
    self.df_train = df_train
    self.df_calendar = df_calendar
    self.exog_fit, self.exog_pred = self.get_exog_variables()
    self.top_level = self.get_top_level()
    self.train_len = len(self.df_train.columns)
    self.epochs = epochs
    self.lookback = lookback

  def get_top_level(self):
    """
    Calculate top level time series by aggregating low level time-series
    """
    data = np.sum(self.df_train.values, axis=0)

    return data

  def get_exog_variables(self):
    self.df_calendar['snap_CA'] = self.df_calendar['snap_CA'].astype(int)
    self.df_calendar['snap_TX'] = self.df_calendar['snap_TX'].astype(int)
    self.df_calendar['snap_WI'] = self.df_calendar['snap_WI'].astype(int)
    calendar = self.df_calendar[['d', 'snap_CA','snap_TX','snap_WI', 'event_name_1', 'event_name_2', 'tm_w_end', 'sell_price', 'tm_w', 'tm_m', 'tm_dw']].copy()
    calendar['snap_count'] = calendar[['snap_CA','snap_TX','snap_WI']].apply(np.sum,axis=1)

    calendar['is_event_1'] = [isinstance(x , str)*1 for x in calendar['event_name_1']]
    calendar['is_event_2'] = [isinstance(x , str)*1 for x in calendar['event_name_2']]
    calendar['is_event'] = calendar[['is_event_1', 'is_event_2']].apply(np.sum, axis=1)
    calendar['is_event'] = np.where(calendar['is_event']>0,1,0)

    exog_fit = calendar[['snap_count', 'is_event', 'tm_w_end', 'sell_price', 'tm_w', 'tm_m', 'tm_dw']].iloc[:TEST_START,:]
    exog_fit.index = range(len(exog_fit))
    exog_pred = calendar[['snap_count', 'is_event', 'tm_w_end', 'sell_price', 'tm_w', 'tm_m', 'tm_dw']].iloc[TRAIN_START:TEST_START+self.horizon,:]
    exog_pred.index = range(len(exog_pred))

    exog_fit = TimeSeries.from_dataframe(exog_fit, value_cols=exog_fit.columns)
    exog_pred = TimeSeries.from_dataframe(exog_pred, value_cols=exog_fit.columns)

    return exog_fit, exog_pred

  def get_weights(self):
    w = np.sum(self.df_train.iloc[:, -28:].values, axis=1) / sum(self.top_level[-28:])
    w = w.reshape(len(w), 1)

    return w

  def predict_top_level(self):
    print(f"training with {self.epochs} epochs and {self.lookback} lb window")
    model = DLinearModel(
      input_chunk_length=self.lookback,
      output_chunk_length=self.horizon,
      n_epochs=self.epochs,
      model_name='dlinear_model',
      random_state=42
    )

    y_train = TimeSeries.from_values(self.get_top_level())

    model.fit(
        y_train,
        past_covariates=self.exog_fit,
        future_covariates=self.exog_pred,
        verbose=False
    )
    preds = model.predict(
        self.horizon,
        past_covariates=self.exog_fit,
        future_covariates=self.exog_pred
    )

    return preds.values()

  def predict_bottom_level(self):
    w = self.get_weights()

    top_level_preds = self.predict_top_level()
    top_level_preds = top_level_preds.reshape(1,len(top_level_preds))
    preds = np.multiply(top_level_preds, w)
    item_ids = self.df_train.index
    res = pd.DataFrame({
        'id': item_ids
    })

    predictions_df = pd.DataFrame(preds, columns=[f'F{i+1}' for i in range(28)])
    preds = pd.concat([res['id'], predictions_df], axis=1)

    preds.set_index('id', inplace=True)

    return preds

In [ ]:
df_train, df_calendar = get_data_by_store_id('CA_1', 'HOBBIES_1')
df_train

d,1,2,3,4,5,6,7,8,9,10,...,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_008_CA_1_evaluation,12.0,15.0,0.0,0.0,0.0,4.0,6.0,5.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_009_CA_1_evaluation,2.0,0.0,7.0,3.0,0.0,2.0,3.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_010_CA_1_evaluation,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_012_CA_1_evaluation,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_015_CA_1_evaluation,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOBBIES_1_125_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_232_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_269_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
model = M5DLinear(horizon=28, df_train=df_train.iloc[:, 0:1941].copy(), df_calendar=df_calendar, epochs=100, lookback=56)

In [ ]:
df_calendar

,d,event_name_1,event_name_2,snap_CA,snap_TX,snap_WI,tm_w_end,tm_dw
d,,,,,,,,
1,1,NaN,NaN,0,0,0,1,5
2,2,NaN,NaN,0,0,0,1,6
3,3,NaN,NaN,0,0,0,0,0
4,4,NaN,NaN,1,1,0,0,1
5,5,NaN,NaN,1,0,1,0,2
...,...,...,...,...,...,...,...,...
1965,1965,NaN,NaN,0,1,1,0,2
1966,1966,NaN,NaN,0,0,0,0,3
1967,1967,NaN,NaN,0,0,0,0,4


In [ ]:
def train_model(lookback, epochs, model_name):
  submission = pd.read_csv('/content/drive/MyDrive/data/m5-data/sample_submission.csv')
  submission.set_index('id', inplace=True)
  for store_id in stores:
    for dept_id in depts:
      df_train, df_calendar = get_data_by_store_id(store_id, dept_id)
      model = M5DLinear(horizon=28, df_train=df_train.iloc[:, 0:1941].copy(), df_calendar=df_calendar, epochs=epochs, lookback=lookback)
      preds = model.predict_bottom_level()

      submission.update(preds)

      del df_train
      del df_calendar
      del model
      gc.collect()

  submission.reset_index(inplace=True)
  submission.to_csv(f"/content/drive/MyDrive/data/m5-data/submissions/{model_name}.csv", index=False)

In [ ]:
epochs=[100, 150, 200]
lookbacks=[
    56,
]

for lookback in lookbacks:
  for epoch_n in epochs:
    model_name = f"dlinear-lb-{lookback}-ep-{epoch_n}-{VER}"
    train_model(lookback, epoch_n, model_name)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


training with 100 epochs and 56 lb window


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


training with 200 epochs and 56 lb window


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=200` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:

map = {
    56: 100,
}

for k, v in map.items():
  print(f"{k} and {v}")

7 and 150
14 and 150
28 and 150
56 and 100
112 and 100
